In [1]:
import open3d as o3d
import numpy as np
import math

In [2]:
pcd = o3d.io.read_point_cloud("data/test_capture.ply")
axes = o3d.geometry.TriangleMesh.create_coordinate_frame()
print(pcd)
print(np.asarray(pcd.points))
o3d.visualization.draw_geometries([pcd, axes])

PointCloud with 72198 points.
[[-0.44946289  0.39306641 -0.94238281]
 [-0.4453125   0.39233398 -0.94042969]
 [-0.44165039  0.3918457  -0.93945312]
 ...
 [ 0.45117188 -0.33422852 -0.85693359]
 [ 0.45385742 -0.33422852 -0.85693359]
 [ 0.45678711 -0.33422852 -0.85693359]]


In [3]:
plane_model, inliers = pcd.segment_plane(distance_threshold=0.01,
                                         ransac_n=3,
                                         num_iterations=1000)

[a, b, c, d] = plane_model
pcd = pcd.translate((0,0,d/c))
axes = o3d.geometry.TriangleMesh.create_coordinate_frame()

inlier_cloud = pcd.select_by_index(inliers)
inlier_cloud.paint_uniform_color([1.0, 0, 0])
outlier_cloud = pcd.select_by_index(inliers, invert=True)
o3d.visualization.draw_geometries([inlier_cloud, outlier_cloud, axes])

In [4]:
cos_theta = c / math.sqrt(a**2 + b**2 + c**2)
sin_theta = math.sqrt((a**2+b**2)/(a**2 + b**2 + c**2))
u_1 = b / math.sqrt(a**2 + b**2 )
u_2 = -a / math.sqrt(a**2 + b**2)

rotation_matrix = np.array([[cos_theta + u_1**2 * (1-cos_theta), u_1*u_2*(1-cos_theta), u_2*sin_theta],
                            [u_1*u_2*(1-cos_theta), cos_theta + u_2**2*(1- cos_theta), -u_1*sin_theta],
                            [-u_2*sin_theta, u_1*sin_theta, cos_theta]])

print(rotation_matrix)

[[ 0.99254052 -0.00261864  0.12188706]
 [-0.00261864  0.99908073  0.0427883 ]
 [-0.12188706 -0.0427883   0.99162125]]


In [5]:
pcd.rotate(rotation_matrix)
o3d.visualization.draw_geometries([pcd, axes])

In [10]:
o3d.visualization.draw_geometries([pcd, axes])